**Initialization**
* I use these 3 lines of code on top of my each Notebooks because it will help to prevent any problems while reloading and reworking on a Project or Problem. And the third line of code helps to make visualization within the Notebook.

In [1]:
#@ Initialization:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**Downloading the Dependencies**
* I have downloaded all the Libraries and Dependencies required for this Project in one particular cell.

In [3]:
#@ Downloading the Libraries and Dependencies:
# !pip install nlpia                                                       # Downloading the NLPIA Package.

import numpy as np                                                         # Module for matrix multiplication.
from nlpia.loaders import get_data 
import os, re
from random import shuffle                                                 # Module for shuffling the Dataset.
from IPython.display import display

from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint

**Getting the Data**
* I will use the [**Cornell Movie Dialog Dataset**](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). Using the entire Cornell Movie Dialog Dataset can be computationally intensive because a few sequences have more than 2000 tokens. I will use **NLPIA** Package to load the Cornell Movie Dialog Dataset and I will pre process the Dialog Corpus.

In [4]:
#@ Getting the Data:
df = get_data("moviedialog")                                                # Accessing the Cornell Movie Dialog Corpus.

#@ Processing the Data:
input_texts = []                                                            # The array holds the input text from the Corpus.
target_texts = []                                                           # The array holds the target text from the Corpus.
input_vocabulary = set()                                                    # Holds the seen characters in the input text.
output_vocabulary = set()                                                   # Holds the seen characters in the target txt.

start_token = "\t"                                                          # Target sequence is annotated with start Token.
stop_token = "\n"                                                           # Target sequence is annotated with sop Token.
# max_training_samples = min(25000, len(df) - 1)                              # Defines the lines used for Training.

for input_text, target_text in zip(df.statement, df.reply):
  target_text = start_token + target_text + stop_token                      # The Target Text needs to be wrapped with start and stop tokens.
  input_texts.append(input_text)
  target_texts.append(target_text)
  
  #@ Compiling the Vocabulary set:
  for char in input_text:
    if char not in input_vocabulary:
      input_vocabulary.add(char)
  
  for char in target_text:
    if char not in output_vocabulary:
      output_vocabulary.add(char)

#@ Inspecting the Data:

display(f"Number of samples: {len(input_texts)}")
display(f"Number of unique Input Tokens: {len(input_vocabulary)}")
display(f"Number of unique Output Tokens: {len(output_vocabulary)}")

'Number of samples: 64350'

'Number of unique Input Tokens: 44'

'Number of unique Output Tokens: 46'

**Building the Character Dictionary**
* I will convert each characters of the Input and Target Texts into one hot vectors that represent each characters. In order to generate one hot vectors I will generate token dictionaries where every character is mapped to an index. I will also generate the reverse dictionaries which will be used to convert generated index into characters. 

In [5]:
#@ Sorting the List of Characters:
input_vocabulary = sorted(input_vocabulary)
output_vocabulary = sorted(output_vocabulary)

#@ Calculating the Maximum number of Unique Characters:
input_vocab_size = len(input_vocabulary)
output_vocab_size = len(output_vocabulary)

#@ Determining the Maximum number of Sequence Tokens:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#@ Creating the Token Dictionaries:
input_token_index = dict([(char, i) for i,char in enumerate(input_vocabulary)])
target_token_index = dict([(char, i) for i,char in enumerate(output_vocabulary)])

#@ Creating the Reverse Token Dictionaries:
reverse_input_char_index = dict((i, char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char,i in target_token_index.items())

#@ Inspecting the Data:
display(f"Maximum sequence length for Inputs: {max_encoder_seq_length}")
display(f"Maximum sequence length for Outputs: {max_decoder_seq_length}")

'Maximum sequence length for Inputs: 100'

'Maximum sequence length for Outputs: 102'

**Generating One Hot Encoded Training sets**
* Now, I will convert the input and target text into one hot Encoded Tensors.

In [6]:
#@ Creating character sequence Encoder and Decoder Training Set:

#@ Initializing the Tensors with zeros:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype="float32")

#@ Looping over the Training Samples:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  #@ Looping over each character of each Samples:
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1.
  # decoder_input_data[i, t+1:, target_token_index[" "]] = 1.
  # decoder_target_data[i, t:, target_token_index[" "]] = 1.

### **Sequence to Sequence Chatbot**
* I have completed all the Training set preparations by performing the tasks such as Converting the preprocessed Corpus into Input and Target Samples and creating Index Dictionaries and converting the Samples into One hot Tensors. Now, I will train the Sequence to sequence Chatbot. 

In [8]:
#@ Parameters of LSTM Neural Networks:
batch_size = 64                                 # Number of samples shown to the network before updating the weights.
epochs = 100                                    # Number of times for passing the Training.
num_neurons = 256                               # Setting the number of neuron dimensions to 256.

#@ Sequence to Sequence Encoder Decoder Network:
checkpoint_filepath = "/content/chatbotmodel"
model_checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    monitor = "loss",
    mode = "min",
    save_best_only = True
)

#@ Creating the Thought Encoder using Keras Functional API:
encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)                                          # Returning the internal state of LSTM.
encoder_outputs, state_h, state_c = encoder(encoder_inputs) 
encoder_states = [state_h, state_c]                                                     # First value of LSTM is the Output.

#@ Creating the Thought Decoder using Keras Functional API:
decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(
    num_neurons, return_sequences=True, return_state=True
)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)      # Passing initial state to the LSTM Layer.
decoder_dense = Dense(output_vocab_size, activation="softmax")                          
decoder_outputs = decoder_dense(decoder_outputs)                                        # Passing the output to the Softmax Layer.

#@ Creating the Sequence to Sequence Neural Network Model:
model = Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs
)

#@ Compiling the Sequence to Sequence Neural Network Model:
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",                                                    # Using Categorical Crossentropy.
    metrics=["accuracy"]
)

#@ Training the Sequence to Sequence Neural Network Model:
model.fit(
    [encoder_input_data, decoder_input_data], 
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,                                                              # 10% of Samples are splitted for Validation.
    callbacks = [model_checkpoint_callback]
)

Epoch 1/100
905/905 [==============================] - 25s 27ms/step - loss: 0.7592 - accuracy: 0.1202 - val_loss: 0.6470 - val_accuracy: 0.1546
Epoch 2/100
905/905 [==============================] - 23s 26ms/step - loss: 0.5908 - accuracy: 0.1623 - val_loss: 0.5738 - val_accuracy: 0.1741
Epoch 3/100
905/905 [==============================] - 23s 26ms/step - loss: 0.5381 - accuracy: 0.1762 - val_loss: 0.5385 - val_accuracy: 0.1837
Epoch 4/100
905/905 [==============================] - 23s 26ms/step - loss: 0.5105 - accuracy: 0.1836 - val_loss: 0.5187 - val_accuracy: 0.1891
Epoch 5/100
905/905 [==============================] - 23s 26ms/step - loss: 0.4930 - accuracy: 0.1883 - val_loss: 0.5049 - val_accuracy: 0.1932
Epoch 6/100
905/905 [==============================] - 23s 26ms/step - loss: 0.4806 - accuracy: 0.1917 - val_loss: 0.4965 - val_accuracy: 0.1952
Epoch 7/100
905/905 [==============================] - 23s 26ms/step - loss: 0.4712 - accuracy: 0.1942 - val_loss: 0.4908 - val_ac

**Saving the Sequence to Sequence Chatbot Model**

In [46]:
#@ Saving the Sequence to Sequence Network Model:
model.load_weights(checkpoint_filepath)
model_structure = model.to_json()
with open("chatbot_model.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights("chatbot_model.h5")
print("Model saved successful!!")

model.load_weights("chatbot_model.h5")                         # Loading the saved Model.

Model saved successful!!


**Assembling the Model for Sequence Generation**

In [27]:
#@ Creating the Response Generator Model:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#@ Creating the Model:
decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    outputs=[decoder_outputs] + decoder_states
)

### **Predicting the Sequence**
* I will define a Function for generating the Response of the Chatbot. This Function is the heart of Response Generation of the Chatbot which accepts one hot encoded input sequence, generates the Thought Vector and the Thought Vector generates the appropriate response by using the **Trained Network**. 

In [28]:
#@ Building the Character Based Translator:
def decode_sequence(input_seq):
  #@ Generating the Thought Vector:
  thought = encoder_model.predict(input_seq)

  target_seq = np.zeros((1, 1, output_vocab_size))                          # Initializing it as a Zero Tensor.
  target_seq[0, 0, target_token_index[stop_token]] = 1.                     # First Input Token to the decoder is the input token.

  stop_condition = False
  generated_sequence = ""
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + thought)     # Passing the generated Token and latest state to the Decoder.
    generated_token_idx = np.argmax(output_tokens[0, -1, :])
    generated_char = reverse_target_char_index[generated_token_idx]
    generated_sequence += generated_char
    if (generated_char == stop_token or 
        len(generated_sequence) > max_decoder_seq_length):
      stop_condition = True                                                 # Setting the condition to True will stop the Loop.
    
    target_seq = np.zeros((1, 1, output_vocab_size))
    target_seq[0, 0, generated_token_idx] = 1.
    thought = [h, c]                                                        # Updating the Thought Vector.

  return generated_sequence

**Generating the Response**
* Now, I will define a helper function to convert the Input String into a reply for the Chatbot to use.

In [25]:
class ChatBot:
  negative_responses = ("No", "Not a chance", "Sorry")
  exit_commands = ("Quit", "Pause", "Exit", "Goodbye", "Bye", "Later", "Stop")

#@ Starting the Conversation:
  def start_chat(self):
    user_response = input("Hi, I'm a T2 Chatbot. Would you like to chat with me?\n")
    if user_response in self.negative_responses:
      print("Okay, Have a great day!")
      return
    self.chat(user_response)

#@ Conversation:
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply) + "\n")
    
  #@ Convert user input into matries:
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros((1, max_encoder_seq_length, input_vocab_size), 
                                 dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_token_index:
        user_input_matrix[0, timestep, input_token_index[token]] = 1.
    return user_input_matrix
  
  #@ Creating the Response:
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_sequence(input_matrix)
    chatbot_response = chatbot_response.replace("<START>", "")
    chatbot_response = chatbot_response.replace("<END>", "")
    return chatbot_response

#@ Checking the Exit Commands:
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Okay, Have a great day!")
        return True
    return False
  
chatbot = ChatBot()                                                                 # Instantiating the Chatbot.                        
chatbot.start_chat()                                                                # Running the Chatbot.

Hi, I'm a T2 Chatbot. Would you like to chat with me?
No
Okay, Have a great day!


In [29]:
def Response(input_text):
  input_seq = np.zeros((1, max_encoder_seq_length, input_vocab_size), dtype="float32")
  for t, char in enumerate(input_text):
    input_seq[0, t, input_token_index[char]] = 1.
  decoded_sentence = decode_sequence(input_seq)
  print("T2 Reply:", decoded_sentence)

In [30]:
Response("what is the internet?")

T2 Reply: i don't know. i was all set.



In [45]:
Response("do you like coffee?")

T2 Reply: yeah.

